In [4]:
import azureml.core
import pandas as pd
import numpy as np
import logging

from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime

In [5]:
print("This notebook was created using version 1.10.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.10.0 of the Azure ML SDK
You are currently using version 1.6.0 of the Azure ML SDK

In [6]:
subscription_id='f8292d2d-2fe0-426b-ac4b-39dcf0166725'         # ensure it matches your Azure subscription id
resource_group='synapse-rg'      # ensure it matches your resource group name
workspace_name='amlworkspace'       # ensure it matches your Azure Machine Learning workspace name
ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
ws.write_config()
ws = Workspace.from_config()
experiment = Experiment(ws, "Product_valueIndex")

In [7]:
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = 'automl-bikeshareforecasting'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['SKU'] = ws.sku
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

Subscription ID   f8292d2d-2fe0-426b-ac4b-39dcf0166725
Workspace         amlworkspace                        
SKU               Enterprise                          
Resource Group    synapse-rg                          
Location          eastus                              
Run History Name  automl-bikeshareforecasting

In [8]:
datastore = ws.get_default_datastore()
print(datastore)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-d69578ca-aa37-4ba3-92aa-c5aca34bdeb9",
  "account_name": "amlworkspace3032050954",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
amlcompute_cluster_name = "bike-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned

In [10]:
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./bike-no.csv'], target_path = 'dataset/', overwrite = True,show_progress = True)

In [11]:
target_column_name = 'cnt'
time_column_name = 'date'


In [12]:
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, './bike-no.csv')]).with_timestamp_columns(fine_grain_timestamp=time_column_name) 
dataset.take(5).to_pandas_dataframe().reset_index(drop=True)

instant       date  season  yr  ...   windspeed  casual  registered   cnt
0  1       2011-01-01  1       0   ...   0.160446   331     654         985 
1  2       2011-01-02  1       0   ...   0.248539   131     670         801 
2  3       2011-01-03  1       0   ...   0.248309   120     1229        1349
3  4       2011-01-04  1       0   ...   0.160296   108     1454        1562
4  5       2011-01-05  1       0   ...   0.186900   82      1518        1600

[5 rows x 14 columns]

In [13]:
# select data that occurs before a specified date
train = dataset.time_before(datetime(2012, 8, 31), include_boundary=True)
train.to_pandas_dataframe().tail(5).reset_index(drop=True)

instant       date  season  yr  ...   windspeed  casual  registered   cnt
0  605     2012-08-27  3       1   ...   0.128733   989     5928        6917
1  606     2012-08-28  3       1   ...   0.190925   935     6105        7040
2  607     2012-08-29  3       1   ...   0.112562   1177    6520        7697
3  608     2012-08-30  3       1   ...   0.077117   1172    6541        7713
4  609     2012-08-31  3       1   ...   0.168533   1433    5917        7350

[5 rows x 14 columns]

In [15]:
test = dataset.time_after(datetime(2012, 9, 1), include_boundary=True)
test.to_pandas_dataframe().head(5).reset_index(drop=True)

instant       date  season  yr  ...   windspeed  casual  registered   cnt
0  610     2012-09-01  3       1   ...   0.113187   2352    3788        6140
1  611     2012-09-02  3       1   ...   0.064071   2613    3197        5810
2  612     2012-09-03  3       1   ...   0.151121   1965    4069        6034
3  613     2012-09-04  3       1   ...   0.236321   867     5997        6864
4  614     2012-09-05  3       1   ...   0.187808   832     6280        7112

[5 rows x 14 columns]

In [16]:
max_horizon = 14

In [17]:
time_series_settings = {
    'time_column_name': time_column_name,
    'max_horizon': max_horizon,    
    'country_or_region': 'US', # set country_or_region will trigger holiday featurizer
    'target_lags': 'auto', # use heuristic based lag setting    
    'drop_column_names': ['casual', 'registered'] # these columns are a breakdown of the total and therefore a leak
}

automl_config = AutoMLConfig(task='forecasting',                             
                             primary_metric='normalized_root_mean_squared_error',
                             blocked_models = ['ExtremeRandomTrees'],                             
                             experiment_timeout_hours=0.3,
                             training_data=train,
                             label_column_name=target_column_name,
                             compute_target=compute_target,
                             enable_early_stopping=True,
                             n_cross_validations=3, 
                             max_concurrent_iterations=4,
                             max_cores_per_iteration=-1,
                             verbosity=logging.INFO,
                            **time_series_settings)

In [29]:
remote_run = experiment.submit(automl_config, show_output=True)
remote_run

Running on remote or ADB.
Running on remote compute: bike-cluster
Parent Run ID: AutoML_cb2a5e65-f4e0-4491-be5e-8186ab2adbd8

Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: ModelSelection. Beginning model selection.
Heuristic parameters: Target_Lag = '[1]'.


****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The time series was analyzed, all data points are aligned with detected frequency.
              

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

TYPE:         Memory Issues Detection
STATUS:       PASSED
DESCRIPTION:  The selected horizon and lag values were analyzed, and no potential memory issues we

In [31]:
remote_run.wait_for_completion()

{'runId': 'AutoML_cb2a5e65-f4e0-4491-be5e-8186ab2adbd8', 'target': 'bike-cluster', 'status': 'Completed', 'startTimeUtc': '2020-08-01T02:35:53.570736Z', 'endTimeUtc': '2020-08-01T02:51:55.456762Z', 'properties': {'num_iterations': '1000', 'training_type': 'TrainFull', 'acquisition_function': 'EI', 'primary_metric': 'normalized_root_mean_squared_error', 'train_split': '0', 'acquisition_parameter': '0', 'num_cross_validation': '3', 'target': 'bike-cluster', 'RawAMLSettingsString': "{'name': 'automl-bikeshareforecasting', 'path': None, 'subscription_id': 'f8292d2d-2fe0-426b-ac4b-39dcf0166725', 'resource_group': 'synapse-rg', 'workspace_name': 'amlworkspace', 'region': 'eastus', 'compute_target': 'bike-cluster', 'spark_service': None, 'azure_service': 'Microsoft.ProjectArcadia', '_local_managed_run_id': None, 'iterations': 1000, 'primary_metric': 'normalized_root_mean_squared_error', 'task_type': 'regression', 'data_script': None, 'validation_size': 0.0, 'n_cross_validations': 3, 'y_min': 

In [32]:
best_run, fitted_model = remote_run.get_output()
fitted_model.steps

[('timeseriestransformer', TimeSeriesTransformer(featurization_config=None, logger=None,
           pipeline_type=<TimeSeriesPipelineType.FULL: 1>)), ('stackensembleregressor', StackEnsembleRegressor(base_learners=[('14', Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=[25, 75], with_centering=False,
       with_scaling=False)), ('decisiontreeregressor', DecisionTreeRegressor(criterion='friedman_mse', max_depth=None,
           max_features=None, max_lea...    min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'))]))],
            meta_learner=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
            training_cv_folds=5))]

In [12]:
datastore = ws.get_default_datastore()
print(datastore)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-d69578ca-aa37-4ba3-92aa-c5aca34bdeb9",
  "account_name": "amlworkspace3032050954",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [33]:
fitted_model.named_steps['timeseriestransformer'].get_engineered_feature_names()

['atemp', 'atemp_WASNULL', 'horizon_origin', 'hum', 'hum_WASNULL', 'instant', 'instant_WASNULL', 'mnth', 'mnth_WASNULL', 'season', 'season_WASNULL', 'temp', 'temp_WASNULL', 'weathersit', 'weathersit_WASNULL', 'weekday', 'weekday_WASNULL', 'windspeed', 'windspeed_WASNULL', 'yr', 'yr_WASNULL', '_automl_target_col_lag1D', 'year', 'year_iso', 'half', 'quarter', 'month', 'day', 'wday', 'qday', 'week', '_IsPaidTimeOff', '_Holiday_1 day after Christmas Day', '_Holiday_1 day after Columbus Day', '_Holiday_1 day after Independence Day', '_Holiday_1 day after Labor Day', '_Holiday_1 day after Martin Luther King, Jr. Day', '_Holiday_1 day after Memorial Day', "_Holiday_1 day after New Year's Day", '_Holiday_1 day after Thanksgiving', '_Holiday_1 day after Veterans Day', "_Holiday_1 day after Washington's Birthday", '_Holiday_1 day before Christmas Day', '_Holiday_1 day before Columbus Day', '_Holiday_1 day before Independence Day', '_Holiday_1 day before Labor Day', '_Holiday_1 day before Martin 

In [34]:
# Get the featurization summary as a list of JSON
featurization_summary = fitted_model.named_steps['timeseriestransformer'].get_featurization_summary()
# View the featurization summary as a pandas dataframe
pd.DataFrame.from_records(featurization_summary)

Dropped      ...       TypeDetected
0   No          ...       Numeric     
1   No          ...       DateTime    
2   No          ...       Numeric     
3   No          ...       Numeric     
4   No          ...       Numeric     
5   No          ...       Numeric     
6   No          ...       Numeric     
7   No          ...       Numeric     
8   No          ...       Numeric     
9   No          ...       Numeric     
10  No          ...       Numeric     
11  No          ...       Numeric     

[12 rows x 5 columns]

In [35]:
test_experiment = Experiment(ws, experiment_name + "_test")

In [36]:
import os
import shutil

script_folder = os.path.join(os.getcwd(), 'forecast')
print(script_folder)
os.makedirs(script_folder, exist_ok=True)
shutil.copy('forecasting_script.py', script_folder)
shutil.copy('forecasting_helper.py', script_folder)

In [50]:
import argparse
import azureml.train.automl
#from azureml.automl.runtime.shared import forecasting_models
from azureml.core import Run
from sklearn.externals import joblib
#import forecasting_helper


parser = argparse.ArgumentParser()
parser.add_argument(
    '--max_horizon', type=int, dest='max_horizon',
    default=10, help='Max Horizon for forecasting')
parser.add_argument(
    '--target_column_name', type=str, dest='target_column_name',
    help='Target Column Name')
parser.add_argument(
    '--time_column_name', type=str, dest='time_column_name',
    help='Time Column Name')
parser.add_argument(
    '--frequency', type=str, dest='freq',
    help='Frequency of prediction')

args = parser.parse_args()
max_horizon = args.max_horizon
target_column_name = args.target_column_name
time_column_name = args.time_column_name
freq = args.freq

run = Run.get_context()
# get input dataset by name
test_dataset = run.input_datasets['test_data']

grain_column_names = []

df = test_dataset.to_pandas_dataframe().reset_index(drop=True)

X_test_df = test_dataset.drop_columns(columns=[target_column_name]).to_pandas_dataframe().reset_index(drop=True)
y_test_df = test_dataset.with_timestamp_columns(None).keep_columns(columns=[target_column_name]).to_pandas_dataframe()

fitted_model = joblib.load('model.pkl')

X_test=X_test_df;				
y_test=y_test_df.values.T[0]
freq='D'




						
"""
    Produce forecasts on a rolling origin over the given test set.
    Each iteration makes a forecast for the next 'max_horizon' periods
    with respect to the current origin, then advances the origin by the
    horizon time duration. The prediction context for each forecast is set so
    that the forecaster uses the actual target values prior to the current
    origin time for constructing lag features.
    This function returns a concatenated DataFrame of rolling forecasts.
"""
df_list = []
origin_time = X_test[time_column_name].min()
while origin_time <= X_test[time_column_name].max():
        # Set the horizon time - end date of the forecast
        horizon_time = origin_time + max_horizon * to_offset(freq)

        # Extract test data from an expanding window up-to the horizon
        expand_wind = (X_test[time_column_name] < horizon_time)
        X_test_expand = X_test[expand_wind]
        y_query_expand = np.zeros(len(X_test_expand)).astype(np.float)
        y_query_expand.fill(np.NaN)

        if origin_time != X_test[time_column_name].min():
            # Set the context by including actuals up-to the origin time
            test_context_expand_wind = (X_test[time_column_name] < origin_time)
            context_expand_wind = (
                X_test_expand[time_column_name] < origin_time)
            y_query_expand[context_expand_wind] = y_test[
                test_context_expand_wind]

        # Make a forecast out to the maximum horizon
        y_fcst, X_trans = fitted_model.forecast(X_test_expand, y_query_expand)

        # Align forecast with test set for dates within the
        # current rolling window
        trans_tindex = X_trans.index.get_level_values(time_column_name)
        trans_roll_wind = (trans_tindex >= origin_time) & (
            trans_tindex < horizon_time)
        test_roll_wind = expand_wind & (
            X_test[time_column_name] >= origin_time)
        df_list.append(align_outputs(y_fcst[trans_roll_wind],
                                     X_trans[trans_roll_wind],
                                     X_test[test_roll_wind],
                                     y_test[test_roll_wind],
                                     target_column_name))

        # Advance the origin time
        origin_time = horizon_time

#   return pd.concat(df_list, ignore_index=True)
	
df_all =  pd.concat(df_list, ignore_index=True)
	
	

file_name = 'outputs/predictions.csv'
export_csv = df_all.to_csv(file_name, header=True)

# Upload the predictions into artifacts
#run.upload_file(name=file_name, path_or_stream=file_name)

In [43]:
import argparse
import azureml.train.automl
from azureml.automl.runtime.shared import forecasting_models
from azureml.core import Run
from sklearn.externals import joblib
import forecasting_helper


parser = argparse.ArgumentParser()
parser.add_argument(
    '--max_horizon', type=int, dest='max_horizon',
    default=10, help='Max Horizon for forecasting')
parser.add_argument(
    '--target_column_name', type=str, dest='target_column_name',
    help='Target Column Name')
parser.add_argument(
    '--time_column_name', type=str, dest='time_column_name',
    help='Time Column Name')
parser.add_argument(
    '--frequency', type=str, dest='freq',
    help='Frequency of prediction')

args = parser.parse_args()
max_horizon = args.max_horizon
target_column_name = args.target_column_name
time_column_name = args.time_column_name
freq = args.freq

run = Run.get_context()
# get input dataset by name
test_dataset = run.input_datasets['test_data']

grain_column_names = []

df = test_dataset.to_pandas_dataframe().reset_index(drop=True)

X_test_df = test_dataset.drop_columns(columns=[target_column_name]).to_pandas_dataframe().reset_index(drop=True)
y_test_df = test_dataset.with_timestamp_columns(None).keep_columns(columns=[target_column_name]).to_pandas_dataframe()

fitted_model = joblib.load('model.pkl')

df_all = forecasting_helper.do_rolling_forecast(
    fitted_model,
    X_test_df,
    y_test_df.values.T[0],
    target_column_name,
    time_column_name,
    max_horizon,
    freq)

file_name = 'outputs/predictions.csv'
export_csv = df_all.to_csv(file_name, header=True)

# Upload the predictions into artifacts
run.upload_file(name=file_name, path_or_stream=file_name)

In [38]:
from run_forecast import run_rolling_forecast

remote_run = run_rolling_forecast(test_experiment, compute_target, best_run, test, max_horizon,
                 target_column_name, time_column_name)
remote_run

In [ ]:
remote_run.wait_for_completion(show_output=False)

In [ ]:
remote_run.download_file('output/predictions.csv', 'predictions.csv')
df_all = pd.read_csv('predictions.csv')

In [39]:

from azureml.automl.core.shared import constants
from azureml.automl.runtime.shared.score import scoring
from sklearn.metrics import mean_absolute_error, mean_squared_error
from matplotlib import pyplot as plt

# use automl metrics module
scores = scoring.score_regression(
    y_test=df_all[target_column_name],
    y_pred=df_all['predicted'],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET))

print("[Test data scores]\n")
for key, value in scores.items():    
    print('{}:   {:.3f}'.format(key, value))
    
# Plot outputs
%matplotlib inline
test_pred = plt.scatter(df_all[target_column_name], df_all['predicted'], color='b')
test_test = plt.scatter(df_all[target_column_name], df_all[target_column_name], color='g')
plt.legend((test_pred, test_test), ('prediction', 'truth'), loc='upper left', fontsize=8)
plt.show()

In [ ]:

from metrics_helper import MAPE, APE
df_all.groupby('horizon_origin').apply(
    lambda df: pd.Series({'MAPE': MAPE(df[target_column_name], df['predicted']),
                          'RMSE': np.sqrt(mean_squared_error(df[target_column_name], df['predicted'])),
                          'MAE': mean_absolute_error(df[target_column_name], df['predicted'])}))

In [ ]:

df_all_APE = df_all.assign(APE=APE(df_all[target_column_name], df_all['predicted']))
APEs = [df_all_APE[df_all['horizon_origin'] == h].APE.values for h in range(1, max_horizon + 1)]

%matplotlib inline
plt.boxplot(APEs)
plt.yscale('log')
plt.xlabel('horizon')
plt.ylabel('APE (%)')
plt.title('Absolute Percentage Errors by Forecast Horizon')

plt.show()